# Práctica 1 - Self-Organising Maps
## Preparación de entorno
#### Importar librerías de código

In [152]:
%reset
# from __future__ import division

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches

%matplotlib inline

Nothing done.


#### Dataset que se va a utilizar para el entrenamiento

In [153]:
# Código para obtener el Dataset que se va a usar en el entrenamiento
valor_min = 0
valor_max = 256     #256 porque el valor máximo es exclusivo, es decir, irá de 0 a 255
valores_color = 3
num_colores = 100
datos = np.random.randint(valor_min, valor_max, (num_colores, valores_color))
print(f"La matriz de 100 colores:\n",datos)







La matriz de 100 colores:
 [[211 136 179]
 [136 153  28]
 [163 180  35]
 [200 210  81]
 [144 170  11]
 [149 185 221]
 [151 196 240]
 [ 64 137  46]
 [ 39  29 198]
 [154 200  52]
 [ 77   8 231]
 [ 38  18   0]
 [ 40   5  31]
 [114 222 213]
 [ 43 202  28]
 [ 33 229 128]
 [154 209   2]
 [122  47 221]
 [ 21  60   4]
 [163  71 139]
 [103  21 205]
 [140  17  21]
 [108  38 167]
 [ 47  68 228]
 [106 126 110]
 [205 171 163]
 [ 44 124 219]
 [ 20 215 133]
 [ 87  38 116]
 [ 82   8  80]
 [ 22 186  53]
 [ 84   7 191]
 [ 32 136  63]
 [ 38 206 164]
 [252  30 112]
 [170  90 213]
 [  6 181 152]
 [227 215  80]
 [130 165 215]
 [136 231 226]
 [ 17 173 234]
 [197  97 202]
 [ 56 203 239]
 [232 152 229]
 [235 167 112]
 [116 138  30]
 [ 34 253  90]
 [190 158  32]
 [253 149   2]
 [229  36  14]
 [138 171 126]
 [132 164  89]
 [ 62 171 137]
 [235 255  75]
 [170 216 123]
 [ 49  12 164]
 [ 21 141  84]
 [164  12 101]
 [238 217 102]
 [175 211 223]
 [ 12  45 131]
 [ 47 246 243]
 [116 178  55]
 [151  59  50]
 [131  43 163

## SOM Setup
#### Variables definidas por el alumno

In [154]:
# Inicializa tamaño del mapa de Kohonen, número de iteraciones y learning rate
# Inicializa normalizar_datos dependiendo de si tienes que normalizar los datos o no
lado_mapa = 10
periodo = 1000
learning_rate = 0.1
normalizar_datos = True

#### A partir de este punto solo hay cálculos. No se introducen más valores "a mano"

In [155]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.

num_entradas = len(datos[0])   #Usamos len para contar el numero de filas y columnas
num_datos = len(datos)     

def obtener_Entrada(datos):
   valor_aleatorio = np.random.randint(0,101)
   return datos[valor_aleatorio]

# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario = lado_mapa/2

# Normaliza los datos si fuese necesario dividiendo cada dato por el máximo en la matriz
# Calculamos el valor maximo de la matriz

valor_maximo = 0
for dato in datos:
    for valor in dato:
        if(valor_maximo < valor):
            valor_maximo = valor  
      
if normalizar_datos:
    datos = datos/valor_max    
    
    
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
#matriz_pesos = np.random.random((dimensiones de la matriz de pesos))

matriz_pesos = np.random.random((lado_mapa*lado_mapa,num_entradas))
print(matriz_pesos)

[[0.95214653 0.49903192 0.39783696]
 [0.85475064 0.2054646  0.17649165]
 [0.54877577 0.7601655  0.52226736]
 [0.25124306 0.42774795 0.31445149]
 [0.50242115 0.59955816 0.85294922]
 [0.96549028 0.12724109 0.66221951]
 [0.85869169 0.09915578 0.26005992]
 [0.86681396 0.15608379 0.92782496]
 [0.09273405 0.84102425 0.381894  ]
 [0.50376167 0.15595837 0.86954664]
 [0.44892839 0.52607478 0.38129112]
 [0.05226531 0.38296374 0.19152376]
 [0.2659738  0.53074431 0.63019319]
 [0.84468718 0.61560882 0.88785838]
 [0.81439516 0.88465678 0.84034027]
 [0.60660271 0.45678351 0.52652807]
 [0.77865314 0.36920863 0.23774551]
 [0.92484928 0.13606427 0.32886549]
 [0.35585034 0.42662352 0.66590419]
 [0.86171341 0.65651047 0.87897859]
 [0.0940234  0.86513765 0.06568651]
 [0.62867782 0.59822611 0.0664893 ]
 [0.01956951 0.8907639  0.98011526]
 [0.7825247  0.19833082 0.79677454]
 [0.60411334 0.94885724 0.35282733]
 [0.55459302 0.22523188 0.70610121]
 [0.55248501 0.48916519 0.24402236]
 [0.02571449 0.64276402 0.18

#### Funciones para entrenar/clasificar

In [156]:
# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""

def calcular_bmu(patron_entrada, matriz_pesos, m):
   
   """
    Diferencias es un array de 3 elementos que tendrá el cuadrado de la resta de la entrada menos el peso para calcular la distancia euclídea que
    terminará calculándose en distancia local haciendo la raíz de la suma de los valores anteriores.
   
   """
   distancia_local = 0                      
   distancia_minima = float('inf')
   diferencias = np.zeros(m)
   bmu = []
   bmu_idx = ()

   for i in range(len(matriz_pesos)):
      for j in range(m):
         diferencias[j] = (patron_entrada[j] - matriz_pesos[i,j]) ** 2
      """

      Distancia local tendrá la distancia que se esté calculando en el momento, es decir, en la primera iteración tendrá la distancia de la 
      neurona 0,0 al patrón de entrada, después la de 0,1 y así sucesivamente.
      Si la distancia local es menor que la distancia mínima consideraremos esa neurona como la bmu. En la primera iteración la primera neurona 
      será la bmu ya que distancia mínima está inicializada a infinito.
      
      """
      distancia_local = np.sqrt(np.sum(diferencias))
      if(distancia_local < distancia_minima):
         distancia_minima = distancia_local
         bmu = matriz_pesos[i]
         bmu_idx = (i//10,i%10)

   # Por último retornamos la bmu y su posición
   return (bmu, bmu_idx)
           
patron_entrada = obtener_Entrada(datos)      
bmu = calcular_bmu(patron_entrada, matriz_pesos, num_entradas)
print(f"El patrón de entrada es:\n",patron_entrada)
print(f"Y la BMU es:")
print(f"Valor:",bmu[0],"\nPosicion:",bmu[1])

    
    

El patrón de entrada es:
 [0.51953125 0.7734375  0.83203125]
Y la BMU es:
Valor: [0.63748652 0.86343477 0.80795927] 
Posicion: (3, 8)


In [157]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
#Aplicamos la función para calcular la variación de lr
def variacion_learning_rate(lr_inicial, i, n_iteraciones):
    return lr_inicial * (1-(i/n_iteraciones)) 

In [158]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
   return 1 + vecindario_inicial * (1-(i/n_iteraciones))

In [159]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU / (2* (vecindario_actual**2)))

#### Funciones para dibujar la salida de la red

In [160]:
# Función para pintar una matriz de valores como colores RGB
def pintar_mapa(matriz_valores):
    fig = plt.figure()
    
    # Establece ejes
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, matriz_pesos.shape[0]+1))
    ax.set_ylim((0, matriz_pesos.shape[1]+1))
    ax.set_title('Self-Organising Map después de %d iteraciones' % periodo)

    # Dibuja los rectángulos de color RGB
    for x in range(1, matriz_valores.shape[0] + 1):
        for y in range(1, matriz_valores.shape[1] + 1):
            ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1,
                         facecolor=matriz_valores[x-1,y-1,:],
                         edgecolor='none'))
    plt.show()

## SOM Entrenamiento

In [161]:
# Entrena la red con el dataset de entrenamiento



## SOM Clasificación

In [162]:
# Clasifica los patrones de entrenamiento con la matriz de pesos recién entrenada



## SOM Prueba

In [163]:
# Clasifica nuevos patrones

